In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from copy import deepcopy
import sys
sys.path.append('../src')
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
# plt.style.use('dark_background')
import mat4py
import data
import pickle as pkl
from viz import *
import viz
import trend_filtering

In [2]:
df = data.get_data(use_processed=True) # all_data=True
df = df[df['valid'] == 1] # exclude test cells, short/long tracks, hotspots
outcome = 'y_consec_thresh'

## TP/FP videos

In [13]:
from array2gif import write_gif

In [5]:
X, Y = {}, {}
for cell_num in range(1, 7):
    X[cell_num], Y[cell_num] = data.get_images(f'Cell{cell_num}_1s')

In [7]:
fp_ids = [2466, 2547, 2577, 2612, 2631, 2667, 2679, 2705, 2732, 2750, 2756, 2776, 2795, 2473, 2435, 2962]
tp_ids = [8988, 3174, 2969, 2966, 2944, 2939, 2937, 2920, 2918, 2910, 2889, 2884, 2874, 2861, 2854, 2851]

In [74]:
for idx in fp_ids:
    i = np.where(df['pid'] == idx)[0][0]
    track = df.iloc[i]
    cell_num = track['cell_num']
    minx, miny, maxx, maxy = min(track['x_pos_seq']), min(track['y_pos_seq']), max(track['x_pos_seq']), max(track['y_pos_seq'])
    clath = X[cell_num][int(track['t']):(int(track['t'])+track['lifetime']), :, :][:, (int(miny)-5):(int(maxy)+5), :][:, :, (int(minx)-5):(int(maxx)+5)]
    clath = 255.0 * (clath - X[cell_num].min()) / (clath.max() - X[cell_num].min())
    clip = np.repeat(np.expand_dims(clath, axis=3), 3, axis=3) # convert to rgb
    write_gif(clip, f'fp_clips/{idx}.gif', fps=5)
    
for idx in tp_ids:
    i = np.where(df['pid'] == idx)[0][0]
    track = df.iloc[i]
    cell_num = track['cell_num']
    minx, miny, maxx, maxy = min(track['x_pos_seq']), min(track['y_pos_seq']), max(track['x_pos_seq']), max(track['y_pos_seq'])
    clath = X[cell_num][int(track['t']):(int(track['t'])+track['lifetime']), :, :][:, (int(miny)-5):(int(maxy)+5), :][:, :, (int(minx)-5):(int(maxx)+5)]
    clath = 255.0 * (clath - X[cell_num].min()) / (clath.max() - X[cell_num].min())
    clip = np.repeat(np.expand_dims(clath, axis=3), 3, axis=3) # convert to rgb
    #clip = 255 * (clip - X[cell_num].min()) / (clip.max() - X[cell_num].min())
    write_gif(clip, f'tp_clips/{idx}.gif', fps=5)

In [61]:
clath = X[cell_num][int(track['t']):(int(track['t'])+track['lifetime']), :, :][:, (int(miny)-5):(int(maxy)+5), :][:, :, (int(minx)-5):(int(maxx)+5)]

# kymographs

In [43]:
def plot_kymograph(df, pids, X, Y):
    
    df = df[df.pid.isin(pids)]
    #df = df[df.lifetime >= 50]
    #df = df[df.y_consec_thresh == 0]
    #cla = np.array([[X[df.cell_num.iloc[i]].mean()]*int(df.t.iloc[i])+ 
    #                 [np.mean(X[df.cell_num.iloc[i]][int(df.t.iloc[i]) + j][int(df.y_pos_seq.iloc[i][j]-1):int(df.y_pos_seq.iloc[i][j]+2),:][:,int(df.x_pos_seq.iloc[i][j])])
    #                 for j in range(df.lifetime.iloc[i])] +
    #                 [X[df.cell_num.iloc[i]].mean()]*(300 - int(df.t.iloc[i]) - df.lifetime.iloc[i]) for i in range(len(df))])
    cla = []
    for i in range(len(df)):
        xmean = X[df.cell_num.iloc[i]].mean()
        xmean = X[1].min()
        t, lt = int(df.t.iloc[i]), df.lifetime.iloc[i]
        x_pos, y_pos = df.x_pos_seq.iloc[i], df.y_pos_seq.iloc[i]
        trace = [xmean]*t + [np.mean(X[df.cell_num.iloc[i]][t+j][int(y_pos[j]-1):int(y_pos[j]+2),:][:,int(x_pos[j]-1):int(x_pos[j]+2)]) for j in range(lt)] + [xmean]*(300 - t - lt)
        cla.append(trace)
    cla = np.array(cla)
    #print(cla.max())
    cla = (cla - X[1].min())/(5572 - X[1].min())
    cla_sparse = []
    for i in range(len(df)):
        cla_sparse.append(cla[i])
        cla_sparse.append([0]*300)
        cla_sparse.append([0]*300)
        cla_sparse.append([0]*300)
    aux = np.array([[Y[1].min()]*int(df.t.iloc[i])+ 
                     [Y[df.cell_num.iloc[i]][int(df.t.iloc[i]) + j, int(df.y_pos_seq.iloc[i][j]), int(df.x_pos_seq.iloc[i][j])]
                     for j in range(df.lifetime.iloc[i])] +
                     [Y[1].min()]*(300 - int(df.t.iloc[i]) - df.lifetime.iloc[i]) for i in range(len(df))])
    #print(aux.max())
    aux = (aux - Y[1].min())/(4045 - Y[1].min())
    aux_sparse = []
    for i in range(len(df)):
        aux_sparse.append([0]*300)
        aux_sparse.append(aux[i])
        aux_sparse.append([0]*300)
        aux_sparse.append([0]*300)
    return np.array([[[cla_sparse[i][j], aux_sparse[i][j], 0] for i in range(len(df)*4)] for j in range(300)])

In [75]:
def plot_kymograph_aligned(df, pids, X, Y):
    
    df = df[df.pid.isin(pids)]
    lmax = max(df.lifetime.values)
    #df = df[df.lifetime >= 50]
    #df = df[df.y_consec_thresh == 0]
    #cla = np.array([[X[df.cell_num.iloc[i]].mean()]*int(df.t.iloc[i])+ 
    #                 [np.mean(X[df.cell_num.iloc[i]][int(df.t.iloc[i]) + j][int(df.y_pos_seq.iloc[i][j]-1):int(df.y_pos_seq.iloc[i][j]+2),:][:,int(df.x_pos_seq.iloc[i][j])])
    #                 for j in range(df.lifetime.iloc[i])] +
    #                 [X[df.cell_num.iloc[i]].mean()]*(300 - int(df.t.iloc[i]) - df.lifetime.iloc[i]) for i in range(len(df))])
    cla = []
    for i in range(len(df)):
        xmean = X[df.cell_num.iloc[i]].mean()
        xmean = X[1].min()
        t, lt = int(df.t.iloc[i]), df.lifetime.iloc[i]
        x_pos, y_pos = df.x_pos_seq.iloc[i], df.y_pos_seq.iloc[i]
        trace = [np.mean(X[df.cell_num.iloc[i]][t+j][int(y_pos[j]-1):int(y_pos[j]+2),:][:,int(x_pos[j]-1):int(x_pos[j]+2)]) for j in range(lt)] + [xmean]*(lmax - lt)
        cla.append(trace)
    cla = np.array(cla)
    print(cla.max())
    cla = (cla - X[1].min())/(7681 - X[1].min())
    cla_sparse = []
    for i in range(len(df)):
        cla_sparse.append(cla[i])
        cla_sparse.append([0]*lmax)
        cla_sparse.append([0]*lmax)
        cla_sparse.append([0]*lmax)
    aux = np.array([[Y[df.cell_num.iloc[i]][int(df.t.iloc[i]) + j, int(df.y_pos_seq.iloc[i][j]), int(df.x_pos_seq.iloc[i][j])]
                     for j in range(df.lifetime.iloc[i])] +
                     [Y[1].min()]*(lmax - df.lifetime.iloc[i]) for i in range(len(df))])
    print(aux.max())
    aux = (aux - Y[1].min())/(6346 - Y[1].min())
    aux_sparse = []
    for i in range(len(df)):
        aux_sparse.append([0]*lmax)
        aux_sparse.append(aux[i])
        aux_sparse.append([0]*lmax)
        aux_sparse.append([0]*lmax)
    return np.array([[[cla_sparse[i][j], aux_sparse[i][j], 0] for i in range(len(df)*4)] for j in range(lmax)])

In [ ]:
kg = plot_kymograph_aligned(df, fp_ids, X, Y)
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(kg, aspect=.2)
ax.set_xticks([])

In [ ]:
kg = plot_kymograph(df, tp_ids, X, Y)
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(kg, aspect=.2)
ax.set_xticks([])

In [72]:
#np.argsort(df.Y_max)[::-1][:50]
tp_ids = df.pid.values[np.argsort(df.Y_max)[::-1][:50]]
kg = plot_kymograph_aligned(df, tp_ids, X, Y)

7680.333333333333
6346


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(kg, aspect=0.5)
ax.set_xticks([])

In [ ]:
tn_ids = df.pid.values[np.argsort(df.Y_max)[:50]]
kg = plot_kymograph_aligned(df, tn_ids, X, Y)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(kg, aspect=0.5)
ax.set_xticks([])